<a href="https://colab.research.google.com/github/Sathishk2/RNN-Play-Generator/blob/main/RNN_Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.92385027e-03  6.24568621e-03 -3.28037445e-03 ...  1.76561484e-03
    2.55341758e-03  1.97769003e-03]
  [-5.17717609e-03 -2.12515984e-03 -4.64547891e-04 ... -2.07733829e-04
    4.85584885e-03  4.46775137e-03]
  [-2.31112749e-03 -3.66190588e-03  3.65154841e-03 ... -1.17615971e-03
    2.62267981e-03  5.64439315e-03]
  ...
  [ 7.63833988e-03 -8.98630917e-03 -3.23810452e-03 ... -4.27521439e-03
    5.92180435e-03 -8.63503478e-03]
  [ 5.78120723e-03 -8.23733583e-03  9.22443112e-04 ... -5.80592221e-03
    1.18069928e-02 -5.77696180e-03]
  [ 8.98790732e-03 -8.70985910e-03  1.85095472e-04 ... -9.99702793e-03
    1.30218305e-02 -3.78036778e-03]]

 [[-7.74423871e-03  4.81611397e-03  8.95929988e-04 ...  2.25606491e-03
   -1.50528853e-04  7.55107217e-03]
  [-1.36672230e-02  7.41811655e-03  1.43003755e-03 ...  4.34147473e-03
    4.61814925e-05  1.31864389e-02]
  [-9.35543608e-03  4.69801528e-03 -1.26730301e-03 ...  9.42492019e-03
    4.25842265e-03  1.32760275e-02]
  ...
  [-7.138

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00192385  0.00624569 -0.00328037 ...  0.00176561  0.00255342
   0.00197769]
 [-0.00517718 -0.00212516 -0.00046455 ... -0.00020773  0.00485585
   0.00446775]
 [-0.00231113 -0.00366191  0.00365155 ... -0.00117616  0.00262268
   0.00564439]
 ...
 [ 0.00763834 -0.00898631 -0.0032381  ... -0.00427521  0.0059218
  -0.00863503]
 [ 0.00578121 -0.00823734  0.00092244 ... -0.00580592  0.01180699
  -0.00577696]
 [ 0.00898791 -0.00870986  0.0001851  ... -0.00999703  0.01302183
  -0.00378037]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.9238503e-03  6.2456862e-03 -3.2803745e-03 -6.0486640e-03
 -9.3368103e-04  8.1168953e-05 -2.7519751e-03 -4.9488340e-04
  5.1312911e-04 -3.0820710e-03  3.0375053e-03 -1.4859792e-03
  5.2219682e-04  9.9566616e-03 -6.3471659e-03  1.8154875e-03
 -6.5121036e-03  4.1176444e-03  9.8372495e-04 -5.2322042e-03
 -1.7319156e-03  7.1699347e-04  1.4836185e-03  2.8640945e-03
 -2.7413322e-03  1.0244120e-03  3.0530379e-03 -1.5895966e-03
 -5.2142814e-03  6.1792997e-04  3.5299871e-03 -3.9780061e-03
  2.2832151e-03  1.4117518e-03  1.3495306e-03  2.4444663e-03
  2.6794146e-03  3.5899952e-03  2.3670685e-03  6.1027021e-03
 -3.1687415e-03  2.6191920e-03  3.1531283e-03  1.0629308e-03
 -2.0730277e-03 -6.6480148e-03 -9.5245655e-04  9.8574031e-03
 -3.8196740e-04  6.5823421e-03  1.3063938e-04  1.7214586e-03
  4.1658292e-03 -2.4801018e-03  1.2754328e-03  1.2953588e-03
  4.5383545e-03  1.7325558e-03 -1.0016391e-02 -2.9886428e-03
 -2.8406043e-04  9.8101940e-05  1.7656148e-03  2.5534176e-03
  1.977690

In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'YJn3,BY:PezmV-o!UI.jblsZ&s.j3PuT$SoneW$ E$jNZtcVXtduNZbKY?viPp;&yE!zTvYwFYFMTaBMYXvExcF!MzPB!?bAEfzm'

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [27]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeot, save me leave,
Since.

DUKE OF AUMERLE:
Fitzwater, thou liest. Is not this in that?

BENVOLIO:
O conscience sounds for joy
To see you talk of a worthy cause,
To make all coburage must not not be seen.
Farewell, old wealous peace.

HARTINGS:
For ever I should be drunk finger, thou shouldst be crown'd?
You Edward shall my father's honourself
But ond, nangelife them weapon have my wailord.
Thou know'st my daughter's masterly shall meet
From Rome Mortession's smiles, and his supposed
his assurance of a wish!

DUKE OF AUMERLE:
Entoness are deceived; e to my master haste.

FRIAR LAURENCE:
You say you do not know the lady's life,
Upon his power to thy answer his grace.

SICINIUS:
Have with thee?

Provost:
None, sir; but get the incip at Barnet first;
And then; and urgently proud out
And marrie
